# Skeleton of Assignment 4:
    test if the distribution of 
    
    1) trip duration of bikers that ride during the day vs night
    
    2) age of bikers for trips originating in Manhattan and in Brooklyn (extra credit)
    
    are different. Use 3 tests: KS, Pearson's, Spearman's. 
    
    Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 
    
    For the KS do the test with the entire dataset and with a subset 200 times smaller
    
    Choose a single significant threshold for the whole exercise. 
    
    For each test phrase the Null Hypothesis in words.
    
    Describe the return of the scipy function you use in each case.
    
    State the result in terms of rejection of the Null.

In [27]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV
import scipy.stats

%pylab inline

import os
# #this makes my plots pretty! but it is totally not mandatory to do it
# import json
# s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
# pl.rcParams.update(s)



Populating the interactive namespace from numpy and matplotlib


# Read in data
I am reading in data from January 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [2]:
datestring = '201501'
getCitiBikeCSV(datestring)

('Downloading', '201501')
file in place, you can continue


In [3]:
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
df.head()

tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St             40.722293   
1             482              W 15 St & 7 Ave             40.739355   
2             505              6 Ave & W 33 St             40.749013   
3             399  Lafayette Ave & St James Pl             40.688515   
4             432            E 7 St & Avenue A             40.726218   

   end station longitude  bikeid    usertype  birth year  gender  
0             -73.991475   18660  Subscriber      1960.0       2  
1             -73.999318   16085  Subscriber      1963.0       1  
2             -73.988484   20845  Subscriber      1974.0       1  
3             -73.964763   19610  Subscriber      1969.0       1  
4             -73.983799   20197  Subscriber      1977.0       1

In [4]:
# df is the dataframe where the content of the csv file is stored
df['date'] = pd.to_datetime(df['starttime'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 
df.head()

tripduration      starttime       stoptime  start station id  \
0          1346  1/1/2015 0:01  1/1/2015 0:24               455   
1           363  1/1/2015 0:02  1/1/2015 0:08               434   
2           346  1/1/2015 0:04  1/1/2015 0:10               491   
3           182  1/1/2015 0:04  1/1/2015 0:07               384   
4           969  1/1/2015 0:05  1/1/2015 0:21               474   

        start station name  start station latitude  start station longitude  \
0          1 Ave & E 44 St               40.750020               -73.969053   
1          9 Ave & W 18 St               40.743174               -74.003664   
2     E 24 St & Park Ave S               40.740964               -73.986022   
3  Fulton St & Waverly Ave               40.683178               -73.965964   
4          5 Ave & E 29 St               40.745168               -73.986831   

   end station id             end station name  end station latitude  \
0             265     Stanton St & Chrystie St             40.722293   
1             482              W 15 St & 7 Ave             40.739355   
2             505              6 Ave & W 33 St             40.749013   
3             399  Lafayette Ave & St James Pl             40.688515   
4             432            E 7 St & Avenue A             40.726218   

   end station longitude  bikeid    usertype  birth year  gender  \
0             -73.991475   18660  Subscriber      1960.0       2   
1             -73.999318   16085  Subscriber      1963.0       1   
2             -73.988484   20845  Subscriber      1974.0       1   
3             -73.964763   19610  Subscriber      1969.0       1   
4             -73.983799   20197  Subscriber      1977.0       1   

                 date  
0 2015-01-01 00:01:00  
1 2015-01-01 00:02:00  
2 2015-01-01 00:04:00  
3 2015-01-01 00:04:00  
4 2015-01-01 00:05:00

In [5]:
df.columns

Index([u'tripduration', u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype', u'birth year', u'gender', u'date'],
      dtype='object')

# SPLIT BY CATEGORY

as an example I am splitting data by gender and looking at age or riders:

**H0: there is no statistical difference in the age distribution of male and female riders**
$$ \alpha = 0.05 $$

extracting the age happens in the cell below

In [6]:
#df is the dataframe where the content of the csv file is stored
df['ageM'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 1)]
df['ageF'] = 2015 - df['birth year'][(df['usertype'] == 'Subscriber') & (df['gender'] == 2)]

In [22]:
#dropping some data I no longer need
df = df.drop([u'starttime', u'stoptime', u'start station id',
       u'start station name', u'start station latitude',
       u'start station longitude', u'end station id', u'end station name',
       u'end station latitude', u'end station longitude', u'bikeid',
       u'usertype'], axis = 1)
df.head()

tripduration  birth year  gender                date  ageM  ageF
0          1346      1960.0       2 2015-01-01 00:01:00   NaN  55.0
1           363      1963.0       1 2015-01-01 00:02:00  52.0   NaN
2           346      1974.0       1 2015-01-01 00:04:00  41.0   NaN
3           182      1969.0       1 2015-01-01 00:04:00  46.0   NaN
4           969      1977.0       1 2015-01-01 00:05:00  38.0   NaN

In [23]:
#your df should look something like this
# look at these data carefully... you may see someinteresting values!
df.describe()

tripduration     birth year         gender           ageM  \
count  285552.000000  279924.000000  285552.000000  227764.000000   
mean      654.325583    1974.951419       1.162485      40.305272   
std       900.775928      11.574637       0.419490      11.595240   
min        60.000000    1899.000000       0.000000      16.000000   
25%       334.000000            NaN       1.000000            NaN   
50%       504.000000            NaN       1.000000            NaN   
75%       772.000000            NaN       1.000000            NaN   
max     43023.000000    1999.000000       2.000000     116.000000   

               ageF  
count  52093.000000  
mean      38.951145  
std       11.405112  
min       17.000000  
25%             NaN  
50%             NaN  
75%             NaN  
max       89.000000

In [24]:
# dropping NaN values
df['ageM'].dropna(inplace= True)
df['ageF'].dropna(inplace= True)

lets split age in 10 year bins. the dataset is very large, so i could be split in smaller bins, but I will chose 10 years in the interest of time. 
the bin size choice should be a balance between properly sample the age space, have enough counts in each bin that the statistical noise is not significant (remember that is > sqrt(N)!) and the computational requirement to computatinal facilities ratio. 

the next several steps are needed if you want to code up the KS test from scratch. that is for extra credit, so if you do not want to do it you may not need to plot split the distribution in bins and create the cumulative HOWEVER it is a great idea to do it anyways to explore your data viaually! remember Ascombe's quartet!!

In [25]:
# plotting histogramswith pandas is a bitdifferent than with pylab
# this is a VERY USEFUL syntaxfor you to knonw!
bins = np.arange(10, 99, 5)
axM = df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).plot(kind='bar', 
                                                                legend=False)
axM.set_title("male riders")
axF = df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).plot(kind='bar',
                                                                legend=False)
axF.set_title("female riders");


**Figure 1: histogrammed distribution of riders' ages by gender **
We need to see the age bins, the x/y axes, and the data trend. We are examing it to get the initial idea about the rider's gender and age characteristics. We should notice that the data size are largely difference which may introduce some bias to the folllowing analysis

this is how the cumulative distributions look like.  Notice that i am normalizing them! if i want to reat an observed distribution like a probablility distribution i have to normalize it

In [26]:
#print df.ageS, df.ageS.cumsum()

csM=df.ageM.groupby(pd.cut(df.ageM, bins)).agg([count_nonzero]).cumsum()

csF=df.ageF.groupby(pd.cut(df.ageF, bins)).agg([count_nonzero]).cumsum()

print (np.abs(csM / csM.max()-csF / csF.max()))

pl.plot(bins[:-1] + 5, csM / csM.max(), label = "M")
pl.plot(bins[:-1] + 5, csF / csF.max(), label = "F")
pl.plot(bins[:-1] + 5, np.sqrt(csF / csF.max() - csM / csM.max())**2, 'k-',
        label = "difference")
pl.xlabel("Age")
pl.ylabel("Normalized Cumulative Number")
pl.legend()

          count_nonzero
ageM                   
(10, 15]       0.000000
(15, 20]       0.001657
(20, 25]       0.010955
(25, 30]       0.060955
(30, 35]       0.065548
(35, 40]       0.052146
(40, 45]       0.031750
(45, 50]       0.018657
(50, 55]       0.011015
(55, 60]       0.010957
(60, 65]       0.008149
(65, 70]       0.003203
(70, 75]       0.000663
(75, 80]       0.000188
(80, 85]       0.000002
(85, 90]       0.000075
(90, 95]       0.000000


** Figure 2: the cumulative distribution of CitiBike riders' ages by gender** We normalized the two datasets to see their general trend. We could notice that the male and female groups are basically following the similar trends and the differences are generally smaller than 10%. There is an increase in normalized cumulative number in the age range (25,40). 

They look similar! But the difference gets to 10%. If I wanted to code the KS test by hand I woud have everything I need: the normalized cumulative distributions can be subtracted from each other and the max distance can calculated. 

Notice that there may be NaN values you are gonna have to deal with! 
You can do that for example with a Boolean statementsuch as  df.ageF[~np.isnan(df.ageF)] or you can use numpy functions that deal with Nan values: nansum, nanmean, nanstd...

lets run the scipy KS test

# KS tests to compare 2 samples

http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

the KS test in scipy returns the p-value BUT make sure you understand what the NULL is! read the documentation carefully! what is the null hypothesis that you can/cannot reject?

In [28]:
ks = scipy.stats.ks_2samp(df.ageM, df.ageF)
print (ks)

Ks_2sampResult(statistic=0.09128675483027056, pvalue=1.1723000718567192e-307)


**  FILL IN THE CELL BELOW!**

In [29]:
## your words here!...
## this cell is for you to tell me what the scipy KS test returned and what it means in terms of NULL HYPOTHESIS
## to do that refer to the scipy documentation to understand the output of the scipy.stats.ks_2samp function

''' 
The result from KS test which is shown above indicates that 
we could REJECT the Null hypothesis that
the male and female rider's datasets are retrieved from a same dataset
at the significance level of 0.05
'''

# In the KS test, the value D is converges to 0, it is the supremum of the set of distances. 
# As p < 0.05  and D is not small, we can reject the null hypotheses.


" \nThe result from KS test which is shown above indicates that \nwe could REJECT the Null hypothesis that\nthe male and female rider's datasets are retrieved from a same dataset\nat the significance level of 0.05\n"

The scipy.stats KS test already tells me the significance and the p-value. 

The next few cells are here just to show you how you would obtain the same result by hand, but they are **not required**. 

Remember: the Null hypothesis is rejected if 

$D_KS(n1,n2) > c(\alpha) \sqrt{\frac{(n1 + n2)}{n1n2}}$

(see class notes) where $c(\alpha$) is the inverse of the KS distribution, and you do not have to know how to get that cause there are tables that list critical values!! 

http://www.real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/kolmogorov-smirnov-test/kolmogorov-distribution/

But also this result depends in your choice of binning through, and thustheresultyou get by hand may not be exactly the same as the one the KS returns. Either way: this is how you would calculate the KS statistics by hand.

In [ ]:
## your words here!...
## this cell is for you to redo the test with reducted dataset 
## and tell me what the scipy ks test returned and what it means in terms of NULL HYPOTHESIS


In [30]:
df_r = df.sample(df['ageM'].shape[0]/200, axis = 0)
df_r['ageM'].dropna(inplace= True)
df_r['ageF'].dropna(inplace= True)
ks_r = scipy.stats.ks_2samp(df_r['ageM'],df_r['ageF'])
print (ks_r)

Ks_2sampResult(statistic=0.1500342495076627, pvalue=0.00094113567980987729)


### In this case, although it is a reduced sample set, we still have p value smaller than 0.05 which enable us to reject the null hypothesis as the previous ks test

# Now retest using a test for correlation. 

That will answer a slightly different question though - formulate the NULL appropriately. The tests for correlations (generally) requires the variable to be paired, so that I can tell if x changes does y change similarly. But the datasets are of different size! You will need to reduce them to the same size. You can do that by subsampling of the data: take only 1 ride every of 200, which you can achieve "slicing and broadcasting" the array or using one of the python function (built in python numpy.random.choice() functions for example: Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

        .. versionadded:: 1.7.0

Parameters
...

But make sure you understand how to use it! there is an option "replace" which you should think about.

# Pearson's  test for correlation

** notice that the Pearson's is a pairwise test: the samples need to be **
 a. the same size
 b. sorted! (how??)
    
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr



### H0: There is NO correlation between male and female riders in each age.
significance level α = 0.05

In [35]:
# your code here!
# wrangle the data as needed
# please perform the Pearson's test 
# and tell me what you find in terms of NULL hypothesis
len_min = min(df['ageM'].shape, df['ageF'].shape)
len_min = len_min[0]
Male_r = np.sort(df['ageM'].sample(n = len_min, axis = 0))
Female_r = np.sort(df['ageF'].sample(n = len_min, axis = 0))

In [36]:
pearson = scipy.stats.pearsonr(Male_r,Female_r)
pearson

(0.9958838927571485, 0.0)

### The correlation coefficient of Pearson's test is 0.99588, which indicate a strong positive linear relationship. The p value is 0.0 and we could reject our null hypothesis. 

# Spearman's  test for correlation

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html#scipy.stats.spearmanr

In [37]:
# your code here!
# wrangle the data as needed
# please perform the Spearman's test and tell me what you find in terms of NULL hypothesis

# scipy.stats.spearmanr(a, b=None, axis=0, nan_policy='propagate')
spearman = scipy.stats.spearmanr(Male_r,Female_r)
spearman

SpearmanrResult(correlation=0.99912898017455842, pvalue=0.0)

** please comment on the numbers that you get in the light of the scipy manual: what is rerutned? what does it mean??**

### The returned value is the correlation coefficient and the corresponding p-value. The correlation coefficient of Spearman's test almost 1, which indicate a strong positive linear relationship. The p value is 0.0 and we could reject our null hypothesis. 